In [ ]:
## Proof of concept: exploring functional genetics in a Jupyter notebook using igap snps


[github conversation](https://github.com/ipython/ipywidgets/issues/838#issuecomment-255506936)|

In [1]:
%%javascript
require.config({
   paths: {cytoscape: 'http://localhost:8099/js/cytoscape-2.7.10'}
   })

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as widgets
import json
from traitlets import Int, Unicode, Tuple, CInt, Dict, validate, observe

class cyjsWidget(widgets.DOMWidget):
    
    _view_name = Unicode('CyjsView').tag(sync=True)
    _view_module = Unicode('cyjs').tag(sync=True)
    frameWidth = Int(400).tag(sync=True)
    frameHeight = Int(300).tag(sync=True)
    msgStringFromPython = Unicode("{}").tag(sync=True)
    msgStringToPython = Unicode("{}").tag(sync=True)
    msg = {};
    status = "initial status message\n"
    selectedNodes = [];
        
    def setSize(self, width, height):
       self.status += "setSize(%d, %d)\n" % (width, height)
       self.frameWidth = width
       self.frameHeight = height
        
    def fit(self, margin=50):
      self.status += "entering fit (%d)\n" % margin
      self.msgStringFromPython = json.dumps({"cmd": "fit", "status": "request",
                                              "callback": "", "payload": margin});
        
    def requestSelectedNodes(self):
      #self.selectedNodes = [];
      self.status += "entering requestSelectedNodes\n";
      self.msgStringFromPython = json.dumps({"cmd": "getSelectedNodes", "status": "request",
                                             "callback": "", "payload": ""});
    def getSelectedNodes(self):
      self.status += "entering getSelectedNodes, current content is %s\n" % self.selectedNodes
      return(self.selectedNodes)

    def selectNodes(self, nodes):
      self.msgStringFromPython = json.dumps({"cmd": "selectNodes", "status": "request",
                                              "callback": "", "payload": nodes});
       
    def clearSelection(self):
      self. msgStringFromPython = json.dumps({"cmd": "clearSelection", "status": "request",
                                              "callback": "", "payload": ""});

        
    @observe('msgStringToPython')
    def msg_arrived(self, change):
        self.status += "---- python - msg arrived\n"
        tmp = change['new']
        self.status += "len of tmp: %d\n" % len(tmp)
        self.status += "type of tmp: %s\n" % type(tmp)
        self.msgStringToPython = tmp
        self.status += "%s\n" % tmp
        self.dispatch(self.msgStringToPython)
 
    def dispatch(self, msgRaw):
        msg = json.loads(msgRaw)
        self.status += "entering dispatch\n"
        self.status += "dispatch this msg: %s\n" % msg
        self.status += "msg.cmd: %s\n" % msg["cmd"]
        if msg["cmd"] == 'storeSelectedNodes':
            self.status += "storing selected nodes to self.selectedNodes %s\n" % msg["payload"]
            self.selectedNodes = msg["payload"]
        elif msg["cmd"] == 'clearCircles':
            self.circles = []
        else:
          print("unknown cmd: %s" % msg["cmd"])
        

In [17]:
%%javascript
"use strict";

require.undef('cyjs');

define('cyjs', ["jupyter-js-widgets", "cytoscape"], function(widgets, cytoscape) {
    
    var CyjsView = widgets.DOMWidgetView.extend({

        initialize: function() {
           this.circles = [];
           this.circleCount = 0;
           this.options = {}; 
           this.msg = "empty in javascript";
           this.msgStringFromPython = "";
           this.defaultHeight = "800px";
           this.defaultWidth = "1000px";
           },

        createDiv: function(){
            var outerDiv = $("<div id='cyOuterDiv' style='border:1px solid gray; height: 800px; width: 1000px'></div>");
            var toolbarDiv = $("<div id='cyToolbarDiv' style='height: 30px; width: 1000px'></div>");
            var cyDiv = $("<div id='cyDiv' style='height: 870px; width: 1000px'></div>");
            outerDiv.append(toolbarDiv);
            outerDiv.append(cyDiv);
            var cyWidget = this;
            var fitButton = $("<button>Fit</button>").click(function(){
                console.log("Fit!");
                console.log("fitButton's notion of this:")
                console.log(cyWidget.cy);
                cyWidget.cy.fit(50);
               });
            toolbarDiv.append(fitButton);
            var fitSelectedButton = $("<button>Fit Selected</button>").click(function(){
                var selectedNodes = cyWidget.cy.filter('node:selected');
                if(selectedNodes.length > 0){
                   cyWidget.cy.fit(selectedNodes, 50);
                   }
               });
            toolbarDiv.append(fitSelectedButton);
            var sfnButton = $("<button>SFN</button>").click(function(){
               cyWidget.cy.nodes(':selected').neighborhood().nodes().select()
               });
            toolbarDiv.append(sfnButton);
            var clearButton = $("<button>Clear</button>").click(function(){
               cyWidget.cy.nodes().unselect();
               cyWidget.cy.edges().unselect();
               });
            toolbarDiv.append(clearButton);
            return(outerDiv);
           },
 
        
        createCanvas: function(){
            var cyjsWidget = this;
            console.log("createCanvas notion of this:")
            console.log(cyjsWidget);
            this.cy = cytoscape({
               container: document.getElementById('cyDiv'),
               //elements: {
               //nodes: [
               //  {data: {id: 'a', name: 'Node A', type: 'big' }},
               //  {data: {id: 'b', name: 'Node B', type: 'little'}},
               //  ],
              //edges: [
              //  {data: {source: 'a', target: 'b'}},
              //         {data: {source: 'b', target: 'a'}}
              // ]},
          ready: function(){
            console.log("small cyjs network ready");
            console.log("ready's notion of this:")
            console.log(this);
            cyjsWidget.cy = this;
            window.cy = this;  // for easy debugging
            console.log("ready's notion of cyjsWidget:")
            console.log(cyjsWidget);
            console.log("calling this.fit")
            //cyWidget.cy.fit(100);
            console.log("--- about to call loadGraph")
            cyjsWidget.loadGraph("network.json");
            cyjsWidget.loadStyle("style.js");
            console.log("    back from loadGraph")
            cyjsWidget.cy.on("select", function(x){
                var selectedNodeCount = cyjsWidget.cy.nodes(":selected").length;
                var selectedEdgeCount = cyjsWidget.cy.edges(":selected").length;
                console.log("selected nodes: " + selectedNodeCount);
                console.log("selected edges:" + selectedEdgeCount);
                });
            cyjsWidget.cy.on("unselect", function(x){
                var selectedNodeCount = cyjsWidget.cy.nodes(":selected").length;
                var selectedEdgeCount = cyjsWidget.cy.edges(":selected").length;
                console.log("selected nodes: " + selectedNodeCount);
                console.log("selected edges:" + selectedEdgeCount);
                });
            } // ready
           })},

        loadStyle: function(filename){
           var cyObj = this.cy;
           console.log("cyjsWidget.loadStyle: " + filename)
           console.log("loadStyle's notion of this:");
           console.log(this);
           console.log("loadStyle's notion of cy:");
           console.log(cyObj);
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("about to getScript: " + url);
           $.getScript(url)
              .done(function(script, textStatus) {
                 console.log(textStatus);
                 cyObj.style(vizmap);
                 })
             .fail(function( jqxhr, settings, exception ) {
                console.log("getScript error trying to read " + filename);
                console.log("exception: ");
                console.log(exception);
                });
          },
        
        loadGraph: function(filename){
           console.log("entering loadGraph");
           var cyObj = this.cy;
              // the robust url of a file in the same directory as the notebook is
              // str.substring(0, str.lastIndexOf("/"));
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("=== about to getScript on " + url);
           $.getScript(url)
              .done(function(script, textStatus) {
                 console.log("getScript: " + textStatus);
                 console.log("nodes: " + network.elements.nodes.length);
                 if(typeof(network.elements.edges) != "undefined")
                    console.log("edges: " + network.elements.edges.length);
                cyObj.add(network.elements);  // no positions yet
                cyObj.nodes().map(function(node){node.data({degree: node.degree()})});
                }) // .done
            .fail(function(jqxhr, settings, exception) {
               console.log("addNetwork getscript error trying to read " + filename);
               });
           },
        
        render: function() { 
            console.log("entering render")
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:frameWidth', this.frameDimensionsChanged, this);
            this.listenTo(this.model, 'change:frameHeight', this.frameDimensionsChanged, this);
            this.listenTo(this.model, 'change:msgStringFromPython', this.dispatchRequest, this);
            var cyjsWidget = this;
            function myFunc(){
               cyjsWidget.createCanvas()
               }
            setTimeout(myFunc, 500);
            },

        dispatchRequest: function(){
           console.log("dispatchRequest");
           var msgRaw = this.model.get("msgStringFromPython");
           var msg = JSON.parse(msgRaw);
           console.log(msg);
           console.log("========================");
           console.log(this);
           switch(msg.cmd) {
              case 'fit':
                 var margin = msg.payload;
                 console.log("fit with margin: " + margin)
                 this.cy.fit(margin);
                 break;
              case 'getSelectedNodes':
                 var selectedNodes = this.cy.filter("node:selected").map(function(node){ 
                     return node.data().id});
                  console.log("-- found these selected nodes: ");
                  console.log(selectedNodes);
                  var jsonString = JSON.stringify({cmd: "storeSelectedNodes",
                                                status: "reply",
                                                callback: "",
                                                payload: selectedNodes})
                  console.log(" *** jsonString: ")
                  console.log(jsonString);
                  this.model.set("msgStringToPython", jsonString);
                  console.log("    after setting 'msgStringToPython");
                  this.touch();
                  break;
               case 'selectNodes':
                  var nodeIDs = msg.payload;
                  console.log("--- selecting these nodes: " + nodeIDs);
                  if(typeof(nodeIDs) == "string")
                     nodeIDs = [nodeIDs];
                 var filterStrings = [];
                 for(var i=0; i < nodeIDs.length; i++){
                   var s = '[id="' + nodeIDs[i] + '"]';
                   filterStrings.push(s);
                   } // for i
                var nodesToSelect = this.cy.nodes(filterStrings.join());
                nodesToSelect.select()
                break;
              case 'clearSelection':
                 this.cy.nodes().unselect();
                 break;
            default:
               console.log("unrecognized msg.cmd: " + msg.cmd);
             } // switch
           console.log("CONCLUDING dispatchRequest")
           }, 
        
        frameDimensionsChanged: function(){
           console.log("frameDimensionsChanged")
           var newWidth  = this.model.get("frameWidth");
           var newHeight = this.model.get("frameHeight");
           console.log("frame: " + newWidth + " x " + newHeight);
           $("#cyOuterDiv").width(newWidth);
           $("#cyOuterDiv").height(newHeight);
           $("#cyToolbarDiv").width(newWidth);
           $("#cyDiv").width(newWidth);
           $("#cyDiv").height(newHeight - $("#cyToolbarDiv").height());
           }, 
        
        events: {
           //"click #svg": "changeHandler"
           }

    });
    return {
        CyjsView: CyjsView
    };
});

<IPython.core.display.Javascript object>

In [18]:
cy = cyjsWidget()
cy

In [5]:
cy.setSize(800, 600)

In [6]:
cy.fit()

In [41]:
import sys, time
sys.path.append("/Users/paul/github/fimoService/client-python")
from FimoClient import *
sys.path.append("/Users/paul/github/getDNAService/client-python")
from GetDNAClient import *

In [42]:
fimo = FimoClient("whovian", 5558)
assert(fimo.getHost() == 'whovian:5558')
dnaService = GetDNAClient("hg38")
assert(dnaService.getSequence("chr1", 1, 5) == 'NNNNN')

 ["Highly recurrent TERT promoter mutations in human melanoma"](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4423787/)
       

In [43]:
sequences = {"tert_wt1": "CCCGGAGGGGG", "tert_wt2": "CCCGGGAGGGG", "tert_mut": "CCCCTTCCGGG"}
fimo.request(sequences)

,#pattern name,matched sequence,p-value,q-value,score,sequence name,start,stop,strand
0,MA0076.2,CCCCTTCCGGG,0.000022,0.000133,13.1818,tert_mut,1,11,+
1,"ELK1,4_GABP{A,B1}.p3",CCCGGAAGGG,0.000045,0.000539,11.7000,tert_mut,2,11,-
2,ETV6_full_2,CCCGGAAGGG,0.000093,0.001120,10.7245,tert_mut,2,11,-
3,MA0645.1,CCCGGAAGGG,0.000095,0.001150,10.7308,tert_mut,2,11,-


In [71]:
cy.requestSelectedNodes()

In [74]:
# time.sleep(5)
print(cy.getSelectedNodes())

['REACTOME_GLYCEROPHOSPHOLIPID_BIOSYNTHESIS']


In [72]:
print(cy.status)

initial status message
entering requestSelectedNodes
---- python - msg arrived
len of tmp: 97
type of tmp: <class 'str'>
{"cmd":"storeSelectedNodes","status":"reply","callback":"","payload":["chr19:40348525-40348525"]}
entering dispatch
dispatch this msg: {'cmd': 'storeSelectedNodes', 'payload': ['chr19:40348525-40348525'], 'callback': '', 'status': 'reply'}
msg.cmd: storeSelectedNodes
storing selected nodes to self.selectedNodes ['chr19:40348525-40348525']
entering fit (50)
entering requestSelectedNodes
---- python - msg arrived
len of tmp: 115
type of tmp: <class 'str'>
{"cmd":"storeSelectedNodes","status":"reply","callback":"","payload":["REACTOME_GLYCEROPHOSPHOLIPID_BIOSYNTHESIS"]}
entering dispatch
dispatch this msg: {'cmd': 'storeSelectedNodes', 'payload': ['REACTOME_GLYCEROPHOSPHOLIPID_BIOSYNTHESIS'], 'callback': '', 'status': 'reply'}
msg.cmd: storeSelectedNodes
storing selected nodes to self.selectedNodes ['REACTOME_GLYCEROPHOSPHOLIPID_BIOSYNTHESIS']



#### load prepared igap snp data

In [73]:
cy.selectedNodes

['REACTOME_GLYCEROPHOSPHOLIPID_BIOSYNTHESIS']

#### Select a SNP in the network view, parse it into a chromosomal location (+/- shoulders), get sequence, run FIMO

In [ ]:
cy.requestSelectedNodes()

In [44]:
snp = cy.getSelectedNodes()[0]
print(snp)

chr19:35292055-35292055


#### define a convenience function to get chrom, start and end from a chromLocString

In [46]:
import re
def parseChromLoc(s):
  regex = re.compile('(chr.*):(\d+)-(\d+)')
  m = regex.match(snp)
  assert(len(m.groups()) == 3)
  (chrom, start, end) = m.groups()
  start = int(start)
  end = int(end)
  return((chrom, start, end))

In [47]:
(chrom, start, end) = parseChromLoc(snp)

In [48]:
shoulder = 6
seq ={"snp1": dnaService.getSequence(chrom, start-shoulder, start+shoulder)}
print(seq)

{'snp1': 'CTCCCCCGGGGGT'}


In [49]:
tbl_fimo = fimo.request(seq)
tbl_fimo

,#pattern name,matched sequence,p-value,q-value,score,sequence name,start,stop,strand
0,MA0524.2,ACCCCCGGGGGA,0.000039,0.000135,11.9898,snp1,2,13,-
1,MA0811.1,ACCCCCGGGGGA,0.000047,0.000166,11.7636,snp1,2,13,-
2,MA0524.2,TCCCCCGGGGGT,0.000067,0.000135,11.1939,snp1,2,13,+
3,MA0810.1,TCCCCCGGGGGT,0.000073,0.000148,11.1455,snp1,2,13,+
4,MA0810.1,ACCCCCGGGGGA,0.000074,0.000148,11.1273,snp1,2,13,-
5,MA0811.1,TCCCCCGGGGGT,0.000083,0.000166,10.9091,snp1,2,13,+


In [50]:
 from igv import IGV, Reference, Track

In [51]:
snpLocus = "%s:%d-%d" % (chrom, start-shoulder, start+ shoulder)
print(snpLocus)

chr19:35292049-35292061


In [52]:
igv = IGV(locus=snpLocus, reference=Reference(id="hg38"), 
          tracks=[Track(
                 name="Genes", 
                 url="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz",
                 indexURL="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz.tbi",
                 display_mode="EXPANDED")])


In [53]:
igv

In [54]:
igv.goto("chr19:35,290,644-35,293,726")

Goto track location 


In [55]:
import pandas as pd

In [56]:
trackTbl = pd.DataFrame([[chrom, start-shoulder, start+shoulder, "a", 17.2449]])
trackTbl.to_csv("lpcat2Snp.bed", sep="\t", header=False, index=False)
newTrack = Track(name="snp1", format="bed", indexed=False, 
                 url="http://localhost:10001/files/cyjs/lpcat2Snp.bed", 
                 display_mode='EXPANDED');

In [57]:
igv.load_track(newTrack)

Loading track into IGV.js


#### simple-minded exploration of snp consequences

In [58]:
print("%s:%d-%d (shoulder: %d)" % (chrom, start, end, shoulder))

chr19:35292055-35292055 (shoulder: 6)


In [59]:
baseSequence = dnaService.getSequence(chrom, start-shoulder, start+shoulder)
print(baseSequence)

CTCCCCCGGGGGT


In [ ]:
mut1Sequences = baseSequence[1]
snpSequence =  dnaService.getSequence(chrom, start-1, start+1)
print(snpSequence)

In [60]:
len(baseSequence)

13

In [61]:
end1 = shoulder
start2 = shoulder + 1
end2 = len(baseSequence)
mutA = baseSequence[0:end1] + 'A' + baseSequence[start2:end2]
mutG = baseSequence[0:end1] + 'G' + baseSequence[start2:end2]
mutT = baseSequence[0:end1] + 'T' + baseSequence[start2:end2]


### easy to commit off-by-one errors, so stack them up for easy comparison

In [62]:
pd.DataFrame([[baseSequence], [mutA], [mutG], [mutT]])

,0
0,CTCCCCCGGGGGT
1,CTCCCCAGGGGGT
2,CTCCCCGGGGGGT
3,CTCCCCTGGGGGT


In [63]:
seqArgs = {"wt": baseSequence, "mutA": mutA, "mutG": mutG, "mutT": mutT}

In [64]:
tbl_fimo = fimo.request(seqArgs)
tbl_fimo

,#pattern name,matched sequence,p-value,q-value,score,sequence name,start,stop,strand
0,MA0524.2,ACCCCCAGGGGA,0.000033,0.000243,12.2041,mutT,2,13,-
1,MA0524.2,ACCCCCGGGGGA,0.000039,0.000243,11.9898,wt,2,13,-
10,MA0810.1,ACCCCCGGGGGA,0.000074,0.000296,11.1273,wt,2,13,-
11,MA0811.1,TCCCCCGGGGGT,0.000083,0.000332,10.9091,wt,2,13,+
12,MA0524.2,TCCCCTGGGGGT,0.000092,0.000294,10.7245,mutT,2,13,+
2,MA0811.1,ACCCCCAGGGGA,0.000040,0.000277,12.0182,mutT,2,13,-
3,MA0524.2,ACCCCCCGGGGA,0.000046,0.000243,11.7653,mutG,2,13,-
4,MA0811.1,ACCCCCGGGGGA,0.000047,0.000277,11.7636,wt,2,13,-
5,MA0811.1,ACCCCCCGGGGA,0.000052,0.000277,11.6364,mutG,2,13,-
6,MA0810.1,ACCCCCCGGGGA,0.000055,0.000296,11.5818,mutG,2,13,-
